## The File for generating HTML tags

In [24]:
from wand.image import Image as wi
import os; from io import BytesIO; import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import time

from tesserocr import PyTessBaseAPI, RIL, iterate_level, PT, OEM

In [25]:
filepath = "../datafiles/dfile.pdf"

In [26]:
pytestapi_path = "C:\\Tesseract\\Tesseract-OCR-v4\\tessdata"

### pdf to Images List

In [27]:
def pdf2ImagesList(filepath):
    pdf = wi(filename = filepath, resolution=800, background = 'white')
    numPages = len(pdf.sequence)   #should be a class prop
    pdfImages = pdf.convert("jpeg")
    
    pageImages = []   #should be a class prop
    for img in pdfImages.sequence:
        page = wi(img)
        PIL_img = Image.open(BytesIO(page.make_blob('jpeg')))
        if(PIL_img.mode != 'RGB'):
            pageImages.append(PIL_img.convert('RGB'))
        else:
            pageImages.append(PIL_img)
    
    return pageImages

In [28]:
filepageImages = pdf2ImagesList(filepath)

In [29]:
len(filepageImages)

4

### Noise calculation [per page]

In [64]:
def get_noise_val_Image(imagePIL, NoiseThreshold_ppage): # NoiseThreshold_ppage is b/w 0 and 1
    '''
    Params:
        imagePIL - PIL image object
        NoiseThreshold_ppage - %age noise threshold for an image range [0, 1]
        
    Returns:
        ratio of dark pixels and total pixels
        boolean page acceptance result based on noise ratio
        binary image with 0 representing noise pixels
    '''
    
    BinaryThreshold = 195  ## threshold per pixel 
    
    grayPIL = imagePIL.convert('L')
    imageNUMPY = np.array(grayPIL, dtype='uint8')
    
    Tessstart = time.time()
    
    with PyTessBaseAPI(path = "C:\\Tesseract\\Tesseract-OCR-v3\\tessdata") as api:
        api.SetImage(imagePIL)
        api.Recognize()
        ri = api.GetIterator()
        level = RIL.TEXTLINE
        for r in iterate_level(ri, level):
            x1, y1, x2, y2 = r.BoundingBox(level)
            imageNUMPY[y1:y2, x1:x2] = 255
    
    bin_ = np.zeros(imageNUMPY.shape)
    bin_ = np.where(imageNUMPY > BinaryThreshold, 1, 0)
    
    pixels_num = bin_.shape[0]*bin_.shape[1]
    ratio = (pixels_num - bin_.sum())/pixels_num
    
    return ratio, ratio > NoiseThreshold_ppage, bin_ #returns the ratio, rejection result, binary_Image

In [67]:
def get_noise_params_PDF(PDFImages, NoiseThreshold_ppage  = 0.50):
    ''' IN-PROGRESS
    PDFImages: r x c x ch x num where num is number of 
    '''
    PAGESCOUNT = len(PDFImages)
    pageNoiseFlag = [0]*PAGESCOUNT   #should be a class prop

    FORstart = time.time()
    
    for i in range(0, len(PDFImages)):
        #ITstart = time.time()
        if(get_noise_val_Image(PDFImages[i], NoiseThreshold_ppage)[1]): #rejection means: page is noisy
            pageNoiseFlag[i] = 1
        #print("Iteration-Time taken", time.time() - ITstart)
    
    print("FOR EndTime taken", time.time() - FORstart)
    
    return   sum(pageNoiseFlag)/PAGESCOUNT, pageNoiseFlag #returns the %age of pages that are noisy [0, 1]

In [68]:
noiseratio, noise_bin = get_noise_params_PDF(filepageImages)

FOR EndTime taken 17.110139846801758


In [69]:
print("Ratio:\t", noiseratio, "\t\tList:\t", noise_bin)

Ratio:	 0.0 		List:	 [0, 0, 0, 0]


### OCR SCAN
* Images to Text Conversion 
* Table Detection (Table detection does happens in the java code but doesn't go in the grain)
     * TH , TR , TD
* Background Detection
    * Specific Line Background
    * Word Level, Block Level & Line Level (Under Progress)
    
* Font Characterstics
    * Font Density ( Tried to do in jar but was inefficent, insufficent and inaccurate)
        * Bold 
        * Light
    * Font Size
* Hand Written Detection (ITS UNDER PROGRESS)
    * HandWritten Model vs Digital Text Model
    * Evaluation of Metrices
    
     
      

In [30]:
def line_bg_detection(bbox, numBIN, NUMPYimage):
    if bbox:
        x1, y1, x2, y2 = bbox
        colorbin = np.ones((numBIN, 3), dtype = 'int16') *-1

        combs = np.where(NUMPYimage[y1:y2, x1:x2, 0] != -1) # Check if the image didn't had the -1 there
        if (combs[0].size != 0): # check if the size is not zero 
            bin_i = 0
            # Choose randomly bins (20 is the default set right now )
            # To check the image or the bin or whatsoever 
            for i in np.random.choice(range(combs[0].shape[0]), numBIN):
                x = combs[1][i]
                y = combs[0][i]
                colorbin[bin_i] = NUMPYimage[y1+y, x1+x, :]
                bin_i += 1
            # Historgram 

            hist = {}
            for r, g, b in colorbin:
                if not (r == -1):
                    RGB = str(r)+"_"+str(g)+"_"+str(b)
                    if RGB in list(hist.keys()):
                        hist[RGB] += 1
                    else:
                        hist[RGB] = 1
            if len(list(hist.keys())):
                return list(hist.keys())[list(hist.values()).index(max(list(hist.values())))].split("_") ## RGB value in list
    return 0

In [31]:
def get_table_row_tags(image):
    image.show()
    with PyTessBaseAPI(path=pytestapi_path) as api:
        api.SetImage(image)
        api.Recognize()
        
        #GET PARA TEXT
        ri = api.GetIterator()
        level = RIL.BLOCK
        
        rows_bbox = []
        for r in iterate_level(ri, level):
            block_type = r.BlockType()
            print(block_type)
            if (block_type == PT.HORZ_LINE):
                print(block_type)
                tmp = {}
                tmp['bbox'] = r.BoundingBox(level)
                rows_bbox.append(tmp)
        
        #GET PARA TEXT
        ri = api.GetIterator()
        level = RIL.WORD
        
        for r in iterate_level(ri, level):
            
            word = r.GetUTF8Text(level)
            curr_bbox = r.BoundingBox(level)
            print(curr_bbox, word) # This is printing the values ! !
            print("HEY HEY HEY",len(rows_bbox))
            for i in range(len(rows_bbox)):
                # Need to test whether hte Rows bbox is working or not ! 
                print(curr_bbox[1] <= rows_bbox[i]['bbox'][1])
                if (curr_bbox[1] <= rows_bbox[i]['bbox'][1]):
                    if 'text' not in list(rows_bbox[i].keys()):
                        rows_bbox[i]['text'] = word
                    else:
                        rows_bbox[i]['text'] += " "+word
                    break
        print(rows_bbox)
        if (len(rows_bbox) > 0):
            table_text = "<table>\n<th>\n"+rows_bbox[0]['text']+'\n</th>\n'
            for i in rows_bbox[1:]:
                if ('text' in list(i.keys())):
                    table_text += "<tr>\n"+i['text']+'\n</tr>\n'
            table_text += "</table> /n "
            print(table_text)
            return table_text
    
        return 0

In [23]:
FileTextBuffer = ""
FileTextBuffer += "<document>\n"

pagecounter  = 1
for page in filepageImages[2:3]:
    print(pagecounter)
    thisPageText = ""
    thisPageText += "<page" +str(pagecounter)+">\n"
    
    with PyTessBaseAPI(path = pytestapi_path) as api:
        api.SetImage(page)
        api.Recognize()
        PILimage = page.copy()
        NUMPYimage = np.array(page, dtype='int16')
        
        ## Preprosessing for background detection --START
        ri = api.GetIterator()
        level = RIL.WORD
        # Word by word iterator 
        for r in iterate_level(ri, level):
            if r:
                bbox = r.BoundingBox(level)
                #print(bbox)
                if bbox: ## Now black (-1) 'em out
                    # Word_BBoxes.append(bbox)
                    # print(bbox)
                    #print(bbox)
                    #print(NUMPYimage.shape)
                    NUMPYimage[bbox[1]:bbox[3], bbox[0]:bbox[2], :] = -1
        ## Preprosessing for background detection --END
        
        ri = api.GetIterator()
        level = RIL.BLOCK # Block based Values 

        # Table is detectable ( we can get the verticle and horizental Lins )
        for r in iterate_level(ri, level):
            if r:
                block_type = r.BlockType() # Type of that specific block
                try:
                    tmp = r.GetUTF8Text(level)

                    #print(r.ParagraphInfo())

                    #print(block_type, tmp)
                    if (block_type is PT.TABLE) or  (block_type == PT.FLOWING_IMAGE):
                        print("yes")
                        table_image = r.GetImage(level, 30, PILimage)
                        if table_image:
                            text_rows = get_table_row_tags(table_image[0])
                            #print(text_rows)
                            if(text_rows):
                                thisPageText += text_rows
                    elif((block_type != PT.UNKNOWN) and (tmp.rstrip() != "")):
                        ###  Backgrpound Color algo -START Params (bbox, numBIN, NumpyImage)
                        rgb_result = line_bg_detection(r.BoundingBox(level), 20, NUMPYimage)
                        ###  Backgrpound Color algo -END
                        if (rgb_result):
                            thisPageText += "<text font_bg = '" + rgb_result[0] + "," + rgb_result[1] + "," + rgb_result[2] +"'>\n" + r.GetUTF8Text(level) + "</text>\n"
                        else:
                            thisPageText += "<text>\n" + r.GetUTF8Text(level) + "</text>\n"
                except:
                    pass
    FileTextBuffer += thisPageText + "</page" +str(pagecounter)+">\n"
    pagecounter += 1

FileTextBuffer += "</document>\n"

1
yes
1
1
8
8
8
1
1
1
9
(47, 182, 302, 248) Linear
HEY HEY HEY 0
(682, 180, 916, 261) faster,
HEY HEY HEY 0
(949, 180, 1134, 269) good
HEY HEY HEY 0
(1163, 180, 1275, 248) for
HEY HEY HEY 0
(1304, 180, 1515, 269) large,
HEY HEY HEY 0
(1548, 180, 1744, 248) fixed
HEY HEY HEY 0
(1988, 180, 2247, 248) unable
HEY HEY HEY 0
(2273, 190, 2347, 248) to
HEY HEY HEY 0
(2376, 180, 2530, 248) deal
HEY HEY HEY 0
(2562, 180, 2729, 248) with
HEY HEY HEY 0
(2755, 180, 3230, 269) non—linearly
HEY HEY HEY 0
(3258, 180, 3621, 268) separable
HEY HEY HEY 0
(3649, 180, 3826, 261) data,
HEY HEY HEY 0
(3936, 180, 4231, 269) [linearly
HEY HEY HEY 0
(4259, 180, 4620, 268) separable
HEY HEY HEY 0
(4651, 180, 4809, 248) data
HEY HEY HEY 0
(4836, 202, 4968, 269) e.g.
HEY HEY HEY 0
(5001, 190, 5143, 248) text
HEY HEY HEY 0
(41, 159, 980, 494) number
HEY HEY HEY 0
(1007, 291, 1088, 359) of
HEY HEY HEY 0
(1109, 291, 1435, 372) features,
HEY HEY HEY 0
(1467, 313, 1561, 359) no
HEY HEY HEY 0
(1990, 291, 2161, 359) does

In [16]:
print(FileTextBuffer)

<document>
<page1>
<text>
— _ We embed the task of identifying bias in the learning phase and iteratively learns from a refined training
data whose potential outliers have been re—labeled.

</text>
<text>
—— _ Finally, we conduct several experiments to show the effectiveness of our algorithm on several real datasets.

</text>
<text>
The rest of this paper is organized as follows: section 2 provides a background and related work on kernels. Sections
3.1 to 3.4 details the proposed semantic kernel while sections 3.4 — 3.6 outlines techniques incorporated to deal with
bias in the data. We provide detailed experimental work and analysis in section 4 and conclude the discussion in this
paper in section 5.

</text>
<text>
. Problem setup and background

</text>
<text>
In this section, we define the problem of classification in the presence of label noise. In traditional supervised
classification, a learner is first trained on a training set whose labels are known and is then used to classify

In [121]:
with open("./output-DOCT_new.xml", 'w+',encoding="utf-8") as f:
    f.write(FileTextBuffer)

In [92]:
FileTextBuffer = ""
FileTextBuffer += "<document>\n"

pagecounter  = 1
for page in filepageImages:
    print(pagecounter)
    thisPageText = ""
    thisPageText += "<page" +str(pagecounter)+">\n"
    
    with PyTessBaseAPI(path = pytestapi_path) as api:
        api.SetImage(page)
        api.Recognize()
        PILimage = page.copy()
        NUMPYimage = np.array(page, dtype='int16')
        
        ## Preprosessing for background detection --START
        ri = api.GetIterator()
        level = RIL.WORD
        # Word by word iterator 
        for r in iterate_level(ri, level):
            if r:
                bbox = r.BoundingBox(level)
                #print(bbox)
                if bbox: ## Now black (-1) 'em out
                    # Word_BBoxes.append(bbox)
                    # print(bbox)
                    #print(bbox)
                    #print(NUMPYimage.shape)
                    NUMPYimage[bbox[1]:bbox[3], bbox[0]:bbox[2], :] = -1
        ## Preprosessing for background detection --END
        
        ri = api.GetIterator()
        level = RIL.BLOCK # Block based Values 

        # Table is detectable ( we can get the verticle and horizental Lins )
        for r in iterate_level(ri, level):
            if r:
                block_type = r.BlockType() # Type of that specific blocl
                tmp = r.GetUTF8Text(level)

                #print(r.ParagraphInfo())
                
                #print(block_type, tmp)
                if (block_type is PT.TABLE) or  (block_type == PT.FLOWING_IMAGE):
                    #print("yes")
                    # table_image = r.GetImage(level, 5, PILimage)
                    
                    # if table_image:
                    #    text_rows = get_table_row_tags(table_image[0])
                    #    #print(text_rows)
                    #   if(text_rows):
                    #        thisPageText += text_rows
                    tmp_text
                elif((block_type != PT.UNKNOWN) and (tmp.rstrip() != "")):
                    ###  Backgrpound Color algo -START Params (bbox, numBIN, NumpyImage)
                    rgb_result = line_bg_detection(r.BoundingBox(level), 20, NUMPYimage)
                    ###  Backgrpound Color algo -END
                    if (rgb_result):
                        thisPageText += "<text font_bg = '" + rgb_result[0] + "," + rgb_result[1] + "," + rgb_result[2] +"'>\n" + r.GetUTF8Text(level) + "</text>\n"
                    else:
                        thisPageText += "<text>\n" + r.GetUTF8Text(level) + "</text>\n"
    FileTextBuffer += thisPageText + "</page" +str(pagecounter)+">\n"
    pagecounter += 1

1
2
3
4


In [ ]:
##